In [1]:
import func
import common as c
import importlib

In [2]:
_ = importlib.reload(func)
_ = importlib.reload(c)


In [3]:
NUM_WORKERS = 4

In [6]:
target = "Front Royal, VA"
# links, text = func.get_seeds(target)
# print(links)
# print("\nText:", text)
link_dict, content_dict = func.scrape_to_depth(target, 1)

In [10]:
c.write_to_file("test_corpus_front_royal_aug_08_2022.pickle", [link_dict, content_dict])

In [4]:
link_dict, content_dict = c.read_from_file("test_corpus_front_royal_aug_08_2022.pickle")

In [5]:
len(content_dict)

284

In [7]:
filtered_content = {i: func.simplify_text(content_dict[i]) for i in content_dict}

In [14]:

sources_ents = {k: func.label_entities(v) for k, v in filtered_content.items()}



In [15]:
print(list(sources_ents.values())[:10])

[[], [('102', 'CARDINAL'), ('E Main Street Front Royal', 'ORG'), ('5406358007', 'DATE'), ('Skyline Drive', 'FAC'), ('the Blue Ridge Mountains', 'ORG')], [('102', 'CARDINAL'), ('E Main Street Front Royal', 'ORG'), ('5406358007', 'DATE'), ('Skyline Drive', 'FAC'), ('the Blue Ridge Mountains', 'ORG')], [('Front Royal', 'ORG'), ('Warren County', 'GPE'), ('Virginia', 'GPE'), ('United States', 'GPE'), ('14,440', 'CARDINAL'), ('2010', 'DATE'), ('Warren', 'GPE'), ('Shenandoah Valley', 'LOC'), ('Front Royal', 'ORG'), ('the Iroquois Confederation', 'ORG'), ('1672', 'DATE'), ('Shawnee', 'PERSON'), ('Iroquois', 'GPE'), ('Cherokee', 'PRODUCT'), ('1721', 'DATE'), ('Iroquois', 'GPE'), ('1744.7', 'CARDINAL'), ('1754', 'DATE'), ('LeHewtown', 'PERSON'), ('European', 'NORP'), ('the 1670s', 'DATE'), ("Chester's Ferry", 'ORG'), ('1736', 'DATE'), ('the 1790s', 'DATE'), ('Helltown', 'GPE'), ('Shenandoah', 'GPE'), ('Front Royal', 'ORG'), ('1788', 'DATE'), ('1854', 'DATE'), ('Orange', 'GPE'), ('Manassas', 'GPE

In [24]:
# sample = list(filtered_content.values())[:5]
# print(sample)
# sents = [s.split(". ") for s in sample]
# sents = [i for j in sents for i in j]
# print("\nSentences:", sents)
# entities = [label_ents(s) for s in sents]
persons = [[e for e in ents if e[1] == "PERSON"] for ents in sources_ents.values()] # get just the entities with a "person" label. 
persons = [i for j in persons for i in j] # flatten the list of persons.
persons = list(filter(lambda x: " " in x[0], persons)) # Do a basic selection for names that have spaces. 

In [25]:
print(persons)

[("Front Royal's", 'PERSON'), ('Charles II', 'PERSON'), ('Bing Crosby', 'PERSON'), ('Frank Nesbitt', 'PERSON'), ('Front Royal', 'PERSON'), ('Chris W. Holloway', 'PERSON'), ('Gary Gillispie', 'PERSON'), ('Christopher Holloway', 'PERSON'), ('Scott Lloyd', 'PERSON'), ('Joseph McFadden', 'PERSON'), ('Jacob Meza', 'PERSON'), ('Bernard Samuels', 'PERSON'), ("James Axler's", 'PERSON'), ('Ryan Cawdor', 'PERSON'), ("Front Royal's", 'PERSON'), ('Charles II', 'PERSON'), ('Bing Crosby', 'PERSON'), ('Frank Nesbitt', 'PERSON'), ('Front Royal', 'PERSON'), ('Chris W. Holloway', 'PERSON'), ('Gary Gillispie', 'PERSON'), ('Christopher Holloway', 'PERSON'), ('Scott Lloyd', 'PERSON'), ('Joseph McFadden', 'PERSON'), ('Jacob Meza', 'PERSON'), ('Bernard Samuels', 'PERSON'), ("James Axler's", 'PERSON'), ('Ryan Cawdor', 'PERSON'), ("Front Royal's", 'PERSON'), ('Charles II', 'PERSON'), ('Bing Crosby', 'PERSON'), ('Frank Nesbitt', 'PERSON'), ('Front Royal', 'PERSON'), ('Chris W. Holloway', 'PERSON'), ('Gary Gilli

### Building a 'Dossier' per Nom

One target data structure is a dictionary of {"Proper Noun": [(src_url_1, text_content_1), (src_url_2, text_content_2), ...]...}. 

I'm thinking of recording the data as CSV's. Since there's a many-to-many relationship between Proper Nouns and sources, we need to factor that relation into three separate files. 
Nouns: (person_ID, "Proper Noun")
Sources: (src_ID, "content")
Mentions: (person_ID, src_ID)


In [56]:
# Use some manually-selected names of people to build something like a dossier.
vetted_persons = [
    "Ryan Cawdor",
    "Bing Crosby",
    "Christopher Holloway",
    "Gary Gillispie",
    "Joseph McFadden",
    "Bernard Samuels",
    "Alexander Spotswood",
    "William Byrd",
    "Abram Strickler",
    "George Bowman",
    "Paul Froman",
    "Maria Magadelena",
    "Robert McKay",
    "Adam Miller",
    "Michael Rinehart",
    "Mathias Selser",
    "Peter Stevens",
    "Samuel Kercehval",
    "Frank Capra",
    "Fred Allen",
    "George Coleman",
    "Bill Morrow",
]

In [57]:
import csv 

def write_to_csv(rows, filename, delimiter=','):
    with open(filename, mode='w') as f:
        w = csv.writer(f, delimiter=delimiter, quotechar='"', quoting=csv.QUOTE_MINIMAL)
        _ = [w.writerow(r) for r in rows]


In [58]:
# persons = {i: [e for e in labeled[i] if e[1]=='PERSON'] for i in labeled}
def assign_ids(seq):
    return [(i, v) for i, v in enumerate(seq)]

ids_persons = assign_ids(vetted_persons)
ids_srcs = assign_ids(content_dict)
print(ids_srcs)

[(0, 'https://maps.google.com/maps?q=Front+Royal,+VA&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiVkMWGhdH5AhUmK0QIHRSzD7IQ_AUIBigB'), (1, 'https://www.frontroyalva.com/714&sa=U&ved=2ahUKEwiVkMWGhdH5AhUmK0QIHRSzD7IQFnoECAkQAg&usg=AOvVaw0kG59nWmc2VOi5cbEsCM6r'), (2, 'https://www.frontroyalva.com/1&sa=U&ved=2ahUKEwiVkMWGhdH5AhUmK0QIHRSzD7IQFnoECAgQAg&usg=AOvVaw0hlr8H3A-hpVlR6eDnwhaS'), (3, 'https://en.wikipedia.org/wiki/Front_Royal,_Virginia%23History&sa=U&ved=2ahUKEwiVkMWGhdH5AhUmK0QIHRSzD7IQ0gJ6BAgLEAU&usg=AOvVaw0ljQZvqZg5jRcmkhAVwWh8'), (4, 'https://en.wikipedia.org/wiki/Front_Royal,_Virginia%23Geography&sa=U&ved=2ahUKEwiVkMWGhdH5AhUmK0QIHRSzD7IQ0gJ6BAgLEAY&usg=AOvVaw0gYPp5Fsf3NLr2zk9TJ-5-'), (5, 'https://en.wikipedia.org/wiki/Front_Royal,_Virginia%23Arts_and_culture&sa=U&ved=2ahUKEwiVkMWGhdH5AhUmK0QIHRSzD7IQ0gJ6BAgLEAc&usg=AOvVaw2ju7BJRgZCNDexLrshg14A'), (6, 'https://en.wikipedia.org/wiki/Front_Royal,_Virginia%23Education&sa=U&ved=2ahUKEwiVkMWGhdH5AhUmK0QIHRSzD7IQ0gJ6BAgLEAg&usg=AOvVaw2K8P-mz35611

In [39]:
# types = [len(v) for v in content_dict.values()]
# _ = [print(k) for k in content_dict.keys()]
# print(types)
content_dict = {k: " ".join(v) for k, v in content_dict.items()}

In [59]:
# ensure all values in content_dict are in fact strings. 
all([isinstance(v, str) for v in content_dict.values()])

True

In [60]:
# for name in persons:
# if name in content:
# return person_id, content_id
mentions = [[(person_id, src_id) for src_id, src in ids_srcs if person.lower() in content_dict[src].lower()] for person_id, person in ids_persons]
mentions = [i for j in mentions for i in j]
mentions = [("person_id", "src_id")] + mentions
print(mentions)

[('person_id', 'src_id'), (0, 3), (0, 4), (0, 5), (0, 6), (0, 76), (1, 3), (1, 4), (1, 5), (1, 6), (1, 49), (1, 76), (1, 85), (2, 3), (2, 4), (2, 5), (2, 6), (2, 76), (3, 3), (3, 4), (3, 5), (3, 6), (3, 76), (4, 3), (4, 4), (4, 5), (4, 6), (4, 76), (5, 3), (5, 4), (5, 5), (5, 6), (5, 76), (6, 41), (7, 41), (8, 41), (9, 41), (10, 41), (11, 41), (12, 41), (13, 41), (14, 41), (15, 41), (16, 41), (18, 49), (19, 49), (20, 49), (21, 49)]


In [61]:
# add column labels to persons and sources.
ids_persons = [("person_id", "person_name")] + ids_persons
ids_srcs = [("src_id", "src_url")] + ids_srcs

In [62]:
filenames = ["persons.csv", "sources.csv", "mentions.csv"]
data_dir = "../data/"
filenames = [data_dir + f for f in filenames]
collections = [ids_persons, ids_srcs, mentions]
file_coll = dict(zip(filenames, collections))
_ = [write_to_csv(v, k) for k, v in file_coll.items()]

In [50]:
import re

# pattern = re.compile(r"[^A-Za-z ]+",)
# A-Za-z0-9 _.,!"'/$
pattern = re.compile(r"[^A-Za-z0-9 _.,!'/$]+",)

def simplify_text(text):
    text = " ".join(text)
#     text = text.lower()
    text = pattern.sub('', text)
    return text